In [7]:
import pandas as pd
from slugify import slugify
import json
import math

def formatINR(number):
    s, *d = str(number).partition(".")
    r = ",".join([s[x-2:x] for x in range(-3, -len(s), -2)][::-1] + [s[-3:]])
    return "".join([r] + d)

In [8]:
# read data

raw_data = pd.read_excel('/home/abhinav/sink/D4D/data.xlsx', engine='openpyxl')
indicators = pd.read_excel('/home/abhinav/sink/D4D/Indicators.xlsx', engine='openpyxl')
dist_meta_data = {"morigaon":{"Title": "Morigaon", "About": "Morigaon district is an administrative district in the state of Assam in India. The district headquarters is located at Morigaon. The ancient place of occult Mayong is located in this district as well as Pobitora Wildlife Sanctuary.", "link": "https://morigaon.gov.in/"}, "nagaon":{"Title": "Nagaon", "About": "Nagaon is an administrative district in the Indian state of Assam. The undivided district was the most populous in the state as per 2011 census. The eastern, western, and southern fringes of the newly organized district were once ruled by different small feudal kings or their agents. It was a newly organized village system-hence called “Nagaon”, Na meaning new.", "link": "https://nagaon.assam.gov.in/"} }
dept_meta_data = {'panchayat-and-rural-development': {'Title': "Panchayat and Rural Development", 'Desc':'The Panchayat & Rural Development Department largely works to enhance livelihood opportunities, address chronic poverty, provides social security and works for economic inclusion of rural poor families. The Panchayat & Rural Development Department has been implementing a number of programmes in the rural area of the State by empowering rural poor through power of rights based law, aiming to transform the geography of poverty in the State.', 'schemes':['mgnrega']}, 
             'public-health-engineering': {'Title': "Public Health Engineering", 'Desc':'The Public Health Engineering Department has a mission to enhance quality of life of the people by ensuring sustainable safe drinking water and sanitation facilities and services along with promoting hygiene practices according to their choices and affordability.', 'schemes':['jjm']}}
scheme_meta_data = {"mgnrega": {"Name": "Mahatma Gandhi National Rural Employment Guarantee Act (MGNREGA)", "Title": "MGNREGA", "Desc": "The objective of MGNREGS is to enhance livelihood security of the rural masses. The scheme is meant to provide at least 100 days of wage employment per financial year (FY) to every rural household whose adult members are willing to do unskilled manual work. This is one of the world’s largest public sector employment programmes that provides guaranteed income through employment.", "dept_slug":"panchayat-and-rural-development", "logo": "/logo/mgnrega.webp"},
                    "jjm": {"Name": "Jal Jeevan Mission (JJM)", "Title": "JJM", "Desc": "The National Rural Drinking Water Programme / Jal Jeevan Mission is an initiative of the Government of India under the Ministry of Jal Shakti. It aims to provide safe and adequate drinking water to all households in rural India through provision of individual household tap connections by the year 2024.", "dept_slug":"public-health-engineering", "logo": "/logo/nhm.png"}}

In [9]:
#Merge indicator data into data file 
data = pd.merge(raw_data,indicators[['indicator_slug','indicator_type', 'indicator_selected', 'indicator_overview', 'indicator_explorer', 'indicator_dept', 'indicator_district', 'indicator_unit']],on='indicator_slug', how='left')


In [10]:
# filter data for district page indicators and structure the column values. Not selecting department indicators only since idnicator values need to be derived using another indicators
# data = data[data.indicator_selected == 'yes']

def get_dept_slug(x):
    
    for key in dept_meta_data:
        if x in dept_meta_data[key]['schemes']:
            return key
        
def get_dept_name(x):
    
    for key in dept_meta_data:
        if x in dept_meta_data[key]['schemes']:
            return dept_meta_data[key]["Title"]      
    

data['district_name'] = data['district_name'].map(lambda x: x.title())
data['district_slug'] = data['district_name'].map(lambda x: slugify(x))
data['block_name'] = data['block_name'].map(lambda x: x.title())
data['block_slug'] = data['block_name'].map(lambda x: slugify(x))
data['scheme_slug'] = data['scheme_name'].map(lambda x: slugify(x))

data['dept_slug'] = data['scheme_slug'].map(lambda x: get_dept_slug(x))
data['dept_name'] = data['scheme_slug'].map(lambda x: get_dept_name(x))

data.head()

,db_code,district_name,district_code,block_name,block_code,financial_year,scheme_name,indicator_report,date_of_update,indicator_name,...,indicator_overview,indicator_explorer,indicator_dept,indicator_district,indicator_unit,district_slug,block_slug,scheme_slug,dept_slug,dept_name
0,27--1,Nagaon,27,Khagorijan,1,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of jobcards applied for,...,no,no,no,no,number,nagaon,khagorijan,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
1,27--2,Nagaon,27,Batadraba,2,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of jobcards applied for,...,no,no,no,no,number,nagaon,batadraba,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
2,27--3,Nagaon,27,Juria,3,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of jobcards applied for,...,no,no,no,no,number,nagaon,juria,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
3,27--4,Nagaon,27,Kaliabor,4,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of jobcards applied for,...,no,no,no,no,number,nagaon,kaliabor,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development
4,27--5,Nagaon,27,Rupahihut,5,2023-2024,MGNREGA,Category Wise Household Workers,7-2023-09-34,No of jobcards applied for,...,no,no,no,no,number,nagaon,rupahihut,mgnrega,panchayat-and-rural-development,Panchayat and Rural Development


In [5]:
# calculate department aggregate values for schemes on department page 
def f(x):
    d = {}
    if x.iloc[0]['scheme_slug'] == "mgnrega":
        d['awppd'] = round (x[x['indicator_slug'] == 'awppd']['indicator_value'].mean(), 2)
        d['cppdy'] = round (x[x['indicator_slug'] == 'cppdy']['indicator_value'].mean(), 2)
        d['tpdue'] = round (x[x['indicator_slug'] == 'tpdue']['indicator_value'].sum(), 2)
        d['pdcpd'] = round(100 * (x[x['indicator_slug'] == 'npdgn']['indicator_value'].sum() / x[x['indicator_slug'] == 'cpmar']['indicator_value'].sum()), 2)
    elif x.iloc[0]['scheme_slug'] == "jjm":
        d['fhtcb'] = round (x[x['indicator_slug'] == 'fhtcb']['indicator_value'].sum(), 2)
        d['phftc'] = round(100 * (x[x['indicator_slug'] == 'hhsta']['indicator_value'].sum() / x[x['indicator_slug'] == 'hhapr']['indicator_value'].sum()), 2)
        d['psdwc'] = round(100 * (x[x['indicator_slug'] == 'dwttc']['indicator_value'].sum() / x[x['indicator_slug'] == 'tdise']['indicator_value'].sum()), 2)
        d['phcip'] = round(100 * (x[x['indicator_slug'] == 'nhtot']['indicator_value'].sum() / x[x['indicator_slug'] == 'tothb']['indicator_value'].sum()), 2)
    else:
        pass
    
    d['last_updated'] = x.iloc[0]['date_of_update']
        
    return pd.Series(d, index=['last_updated','awppd', 'cppdy', 'tpdue', 'pdcpd','fhtcb', 'phftc', 'psdwc', 'phcip']) 

dept_data = data.groupby(['district_name', 'district_slug', 'dept_name', 'dept_slug', 'scheme_name', 'scheme_slug']).apply(f).reset_index()
dept_data.head()


#Test data cration
# remove in production 
# dept_data = dept_data.append(dept_data.iloc[0], ignore_index=True)
# dept_data['dept_slug'].iloc[35]  = "public-health-engineering"
# dept_data['scheme_slug'].iloc[35]  = "jjm"
# dept_data['notwwjjm'].iloc[35]  = 500

,district_name,district_slug,dept_name,dept_slug,scheme_name,scheme_slug,last_updated,awppd,cppdy,tpdue,pdcpd,fhtcb,phftc,psdwc,phcip
0,Morigaon,morigaon,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-34,233.45,269.38,234.48,51.36,NaN,NaN,NaN,NaN
1,Morigaon,morigaon,Public Health Engineering,public-health-engineering,JJM,jjm,7-2023-16-02,NaN,NaN,NaN,NaN,63183.0,28.29,91.23,0.0
2,Nagaon,nagaon,Panchayat and Rural Development,panchayat-and-rural-development,MGNREGA,mgnrega,7-2023-09-34,233.46,260.70,1369.65,21.82,NaN,NaN,NaN,NaN
3,Nagaon,nagaon,Public Health Engineering,public-health-engineering,JJM,jjm,7-2023-16-02,NaN,NaN,NaN,NaN,176351.0,44.86,99.83,0.0


In [16]:
# Dept Page Generator

base_json = {}

for index, row in dept_data.iterrows():
    
    # insert district
    if  row.district_slug not in base_json:
        base_json[row.district_slug] = {
        "distTitle": row.district_name,
        "depts" :
            {
                row.dept_slug: 
                   {
                        "deptTitle": row.dept_name,
                        "collapsible": {
                        "title": "Department Information",
                        "content": [
                            dept_meta_data[row.dept_slug]["Desc"] if row.dept_slug in dept_meta_data else "Dummy Dept Desc"
                            ]
                        },
                        "listTitle": "Browse Schemes and Surveys",
                        "list": [
                        {
                            "label": scheme_meta_data[row.scheme_slug]["Name"],
                            "href": row.scheme_slug,
                            "image": scheme_meta_data[row.scheme_slug]["logo"],
                            "lastUpdated": "-".join(str(row.last_updated).split('-'))[0:-3],
                            "cards": [
                            {
                                "value": ("INR " + str(formatINR(row.awppd))) if row.scheme_slug == "mgnrega" else formatINR(int(row.fhtcb)),
                                "label": "Average wage per personday" if row.scheme_slug == "mgnrega" else "FHTC tagged with beneficiary"
                            },
                            {
                                "value": ("INR " + str(formatINR(row.cppdy)))  if row.scheme_slug == "mgnrega" else (str(row.phftc) + "%"),
                                "label": "Cost per personday" if row.scheme_slug == "mgnrega" else "Percentage of HH with FHTC"
                            },
                            {
                                "value": ("INR " + str(formatINR(row.tpdue)) + " Lakhs")  if row.scheme_slug == "mgnrega" else (str(row.psdwc) + "%"),
                                "label": "Total Payment Due" if row.scheme_slug == "mgnrega" else "Percentage of schools with drinking water connection",
                            },
                            {
                                "value": str(formatINR(row.pdcpd)) + "%"  if row.scheme_slug == "mgnrega" else (str(row.phcip) + "%"),
                                "label":"Person Days Generated as a share of Cumulative Projection of Person Days" if row.scheme_slug == "mgnrega" else "Percentage of habitations covered with CWPP/IHP",
                            }
                            ]
                        },
                        ]
                    },
            }
            
        }

            
    elif  row.dept_slug not in base_json[row.district_slug]["depts"]:
        
        base_json[row.district_slug]["depts"][row.dept_slug] =  {
                        "deptTitle": row.dept_name,
                        "collapsible": {
                        "title": "Department Information",
                        "content": [
                            dept_meta_data[row.dept_slug]["Desc"] if row.dept_slug in dept_meta_data else "Dummy Dept Desc"
                            ]
                        },
                        "listTitle": "Browse Schemes and Surveys",
                        "list": [
                        {
                            "label": scheme_meta_data[row.scheme_slug]["Name"],
                            "href": row.scheme_slug,
                            "image": scheme_meta_data[row.scheme_slug]["logo"],
                            "lastUpdated": "-".join(str(row.last_updated).split('-'))[0:-3],
                            "cards": [
                            {
                                "value": ("INR " + str(formatINR(row.awppd))) if row.scheme_slug == "mgnrega" else formatINR(int(row.fhtcb)),
                                "label": "Average wage per personday" if row.scheme_slug == "mgnrega" else "FHTC tagged with beneficiary"
                            },
                            {
                                "value": ("INR " + str(formatINR(row.cppdy)))  if row.scheme_slug == "mgnrega" else (str(row.phftc) + "%"),
                                "label": "Cost per personday" if row.scheme_slug == "mgnrega" else "Percentage of HH with FHTC"
                            },
                            {
                                "value": ("INR " + str(formatINR(row.tpdue)) + " Lakhs")  if row.scheme_slug == "mgnrega" else (str(row.psdwc) + "%"),
                                "label": "Total Payment Due" if row.scheme_slug == "mgnrega" else "Percentage of schools with drinking water connection",
                            },
                            {
                                "value": str(formatINR(row.pdcpd)) + "%"  if row.scheme_slug == "mgnrega" else (str(row.phcip) + "%"),
                                "label":"Person Days Generated as a share of Cumulative Projection of Person Days" if row.scheme_slug == "mgnrega" else "Percentage of habitations covered with CWPP/IHP",
                            }
                            ]
                        },
                        ]
                    }
    else:
        pass

        
with open('dept_data.json', 'w') as fp:
    json.dump(base_json, fp, indent=4)